<a href="https://colab.research.google.com/github/pradeepprasad/Machine_Learning_Workspace/blob/master/Fruit_Quality_Check_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below are the pre-processing steps to download data files from Kaggle and moving in colab to specific folder for persistence across multiple session.

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
ls ../content/drive/'My Drive'/kaggle_login_file/

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp ../content/drive/'My Drive'/kaggle_login_file/kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#list datasets
!kaggle datasets list -s 'fruits'

In [0]:
cd /content/drive/My Drive/data/sample_data/

In [0]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification 

In [0]:
!kaggle datasets download -d moltean/fruits 

In [0]:
!mv fruits-fresh-and-rotten-for-classification.zip /content/sample_data/

In [0]:
!mv kaggle.json /content/sample_data/

In [0]:
ls ../content/sample_data/fruits-fresh-and-rotten-for-classification.zip

In [0]:
!unzip '/content/drive/My Drive/data/sample_data/fruits-fresh-and-rotten-for-classification.zip'

In [0]:
!mv dataset /content/kaggle

In [0]:
ls '/content/drive/My Drive/data/sample_data/dataset/train'

**Now we are proceeding to access the data available to us.**

In [0]:
# First, we are going to load the file names and their respective target labels into numpy array! 
from sklearn.datasets import load_files
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

#train_dir = '../input/fruits-360_dataset/fruits-360/Training'
train_dir = '/content/drive/My Drive/data/sample_data/dataset/train'
#test_dir = '../input/fruits-360_dataset/fruits-360/Test'
test_dir = '/content/drive/My Drive/data/sample_data/dataset/test'

def load_dataset_and_rescale(path):
    img_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
    reszd_images = img_datagen.flow_from_directory(path,
                                               target_size = (100, 100),
                                               batch_size = 100,
                                               class_mode = 'categorical')

    #files = [files.resize((100,100), Image.ANTIALIAS) for file in files]
    #pp ends for resizing images
    #files = np.array(reszd_images['filenames'])
    #targets = np.array(data['target'])
    #target_labels = np.array(data['target_names'])
    #return files,targets,target_labels
    return reszd_images
    
train_set = load_dataset_and_rescale(train_dir)
test_set = load_dataset_and_rescale(test_dir)
#x_train, y_train,target_labels = load_dataset(train_dir)
#x_test, y_test,_ = load_dataset(test_dir)
#print('Loading complete!')
#print('Training set size : ' , x_train.shape[0])
#print('Testing set size : ', x_test.shape[0])
print('Loading and scaling complete!')

In [0]:
# now images are reshaped to 100*100,RGB format
train_set.image_shape
test_set.image_shape

In [0]:
#now lets get how many classes we have.
no_of_classes = train_set.num_classes
no_of_classes

In [0]:
#below gives total number of data in training set
train_set.n

In [0]:
#Simple CNN from scratch - we are using 3 Conv layers followed by maxpooling layers.
# At the end we add dropout, flatten and some fully connected layers(Dense).

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(no_of_classes,activation = 'softmax'))
model.summary()